## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [21]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [22]:
# Look at APPLICATION_TYPE value counts and adjust bins
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [23]:
# Decrease the number of values for each bin by lowering the cutoff
cutoff = 200  # Adjusted to decrease the number of values for each bin
value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = value_counts[value_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()



APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [24]:
# Look at CLASSIFICATION value counts and adjust bins
application_df['CLASSIFICATION'].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [25]:
# Decrease the number of values for each bin by lowering the cutoff
cutoff = 500  # Adjusted to decrease the number of values for each bin
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_value_counts[classification_value_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION']. value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)


In [27]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies['IS_SUCCESSFUL'].values
X = application_df_dummies.drop('IS_SUCCESSFUL', axis=1).values


In [28]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
# Create a StandardScaler instance
scaler = StandardScaler()


In [30]:
# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [31]:
# Define the model with an increased number of neurons and different activation functions
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 = 120  # Increased number of neurons
hidden_nodes_layer2 = 60   # Increased number of neurons

nn = tf.keras.models.Sequential()

In [32]:
# First hidden layer with 'relu' activation
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer with 'tanh' activation
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='tanh'))

# Output layer with 'sigmoid' activation for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\gonzh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 120)                 │           5,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 60)                  │           7,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,721 (49.69 KB)

 Trainable params: 12,721 (49.69 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Compile the model with adam optimizer and binary_crossentropy loss function
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [34]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 676us/step - accuracy: 0.7086 - loss: 0.5857
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step - accuracy: 0.7326 - loss: 0.5485
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - accuracy: 0.7294 - loss: 0.5504
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 677us/step - accuracy: 0.7348 - loss: 0.5473
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 667us/step - accuracy: 0.7354 - loss: 0.5434
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - accuracy: 0.7358 - loss: 0.5441
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 656us/step - accuracy: 0.7291 - loss: 0.5464
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - accuracy: 0.7354 - loss: 0.5438
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - accuracy: 0.7350 - loss: 0.5437
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 677us/step - accuracy: 0.7380 - loss: 0.5417
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - accuracy: 0.7360 - loss: 0.5392
Epoch 12/100
858/85

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 0s - 2ms/step - accuracy: 0.7262 - loss: 0.5621
Loss: 0.5620539784431458, Accuracy: 0.7262390851974487


In [36]:
nn.save('Models/AlphabetSoupCharity_Optimization2.h5')